In [5]:
%matplotlib inline
from impala.dbapi  import connect
from sqlalchemy import *  
from sqlalchemy.sql import select  
from sqlalchemy.schema import *  
import numpy as np
import pandas as pd
from sqlalchemy.orm import sessionmaker
import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
def ToDisk(filename,query):
    first=query.next()
    first.to_csv(filename,encoding="utf-8")
    for line in query:
        line.to_csv(filename,header=False,mode="a+",encoding="utf-8")

In [7]:
conn=connect(host="192.168.61.170",port=21050,user="yudang.wei",password="19900410w",auth_mechanism="GSSAPI")

In [10]:
gdl_cust='''select custorm_id,hour(call_time) as call_hour,type,sum(cast(time as float)) as hour_sum ,
sum(cast(time as float))/count(time)  as hour_mean,count(time) as hour_cnt from  (select * from data_basic.wyd_app_call_record_hive 
where cast(time as float)>10 and cast(time as float)<7201 and cast(type as float) <4 and  cast(type as float)>=1) d  group by  custorm_id,type,call_hour'''
query=pd.read_sql_query(gdl_cust,conn,chunksize=10000)
%time ToDisk('MSN/app_call_hour_stats_0302.csv',query)

CPU times: user 6min 56s, sys: 3.02 s, total: 6min 59s
Wall time: 7min 33s


In [11]:
gdl_cust='''select custorm_id,dayofweek(call_time) as call_day,type,sum(cast(time as float)) as day_sum ,
sum(cast(time as float))/count(time)  as day_mean,count(time) as day_cnt from  (select * from data_basic.wyd_app_call_record_hive 
where cast(time as float)>10 and cast(time as float)<7201 and cast(type as float) <4  and cast(type as float)>=1) d  group by  custorm_id,type,call_day'''
query=pd.read_sql_query(gdl_cust,conn,chunksize=10000)
%time ToDisk('MSN/app_call_day_stats_0302.csv',query)

CPU times: user 2min 47s, sys: 1.39 s, total: 2min 48s
Wall time: 3min 2s


In [12]:
gdl_cust='''select custorm_id,month(call_time) as call_month,type,sum(cast(time as float)) as month_sum ,
sum(cast(time as float))/count(time)  as month_mean,count(time) as month_cnt from  (select * from data_basic.wyd_app_call_record_hive 
where cast(time as float)>10 and cast(time as float)<7201 and cast(type as float) <4 and cast(type as float)>=1) d  group by  custorm_id,type,call_month'''
query=pd.read_sql_query(gdl_cust,conn,chunksize=10000)
%time ToDisk('MSN/app_call_month_stats_0302.csv',query)

CPU times: user 1min 36s, sys: 794 ms, total: 1min 37s
Wall time: 1min 45s


In [13]:
gdl_cust='''select to_date(cast(call_time as timestamp)) as some_day ,count(time) as time_cnt,sum(cast(time as float))/count(time) as time_mean,
sum(cast(time as float)) as  time_sum  from (select * from data_basic.wyd_app_call_record_hive 
where cast(time as float)>10 and cast(time as float)<7201 and cast(type as float) <4 and cast(type as float)>=1) d  group by  some_day'''
query=pd.read_sql_query(gdl_cust,conn,chunksize=10000)
%time ToDisk('MSN/app_call_some_day_stats_0302.csv',query)

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 60.2 ms


In [14]:
gdl_cust='''select * from  data_basic.wyd_app_call_record_hive   where  custorm_id in ('1492775','3943297','3750461')'''
query=pd.read_sql_query(gdl_cust,conn,chunksize=10000)
%time ToDisk('MSN/app_call_sample.csv',query)

CPU times: user 618 ms, sys: 15 ms, total: 633 ms
Wall time: 3.33 s


In [5]:
year_month_day_hour='''select custorm_id, year(call_time) as call_year,month(call_time) as call_month ,day(call_time) as call_day ,hour(call_time) as call_hour,
type,sum(cast(time as float)) as hour_sum , sum(cast(time as float))/count(time)  as hour_mean,count(time) as hour_cnt from  (select * from data_basic.wyd_app_call_record_hive 
where cast(time as float)>10 and cast(time as float)<3600 and cast(type as float) <3 and  cast(type as float)>=1) d 
group by  custorm_id,type,call_year,call_month,call_day,call_hour'''
query=pd.read_sql_query(year_month_day_hour,conn,chunksize=10000)
%time ToDisk('MSN/year_month_day_hour.csv',query)

CPU times: user 1h 9min 48s, sys: 29.5 s, total: 1h 10min 17s
Wall time: 1h 15min 46s


### 时均值特征

In [10]:
year_month_day_hour='''select custorm_id,call_hour,sum(hour_sum)/count(hour_sum) as hour_sum,sum(hour_cnt)/count(hour_cnt) as hour_cnt,sum(hour_mean)/count(hour_mean) as hour_mean,
sum(case when type='2' then hour_sum end )/sum(hour_sum) as hour_time_out_rate,sum(case when type='2' then hour_cnt end)/sum(hour_cnt) as hour_cnt_out_rate
from data_basic.wyd_call_record_ymdh  group by custorm_id, call_hour'''
query=pd.read_sql_query(year_month_day_hour,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/hourOfDay.csv',query)   ### 时均值，在每天不同的时间平均通话时长，通话频率,每次平均通话时长，时长呼出比，频率呼出比

CPU times: user 3min 45s, sys: 1.99 s, total: 3min 47s
Wall time: 4min 8s


### 日均值特征

In [9]:
Day_mean='''select custorm_id,sum(day_sum) as day_sum ,sum(day_cnt) as day_cnt,sum(day_sum)/sum(day_cnt) as mean_time,
sum(case when type='2' then day_sum end)/sum(day_sum) as time_out_rate,sum(case when type='2' then day_cnt end)/sum(day_cnt) as cnt_out_rate
from data_basic.wyd_call_record_date  group by custorm_id,call_time'''
query=pd.read_sql_query(Day_mean,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/MeanOfDay.csv',query)   ### 日均值，平均每天通话时长，通话频率，平均每次通话时长,时长呼出比，频率呼出比

CPU times: user 11min 40s, sys: 5.82 s, total: 11min 46s
Wall time: 12min 48s


###  不同日历日特征

In [11]:
weekday_mean='''select custorm_id,dayofweek(call_time) as weekday,sum(day_sum)/count(day_sum) as weekday_sum,sum(day_cnt)/count(day_cnt) as weekday_cnt,sum(mean_time)/count(mean_time) as mean_time,
sum(time_out_rate)/count(time_out_rate) as weekday_time_out_rate,sum(cnt_out_rate)/count(cnt_out_rate) as weekday_cnt_out_rate  from
(select custorm_id,call_time,sum(day_sum) as day_sum ,sum(day_cnt) as day_cnt,sum(day_sum)/sum(day_cnt) as mean_time,
sum(case when type='2' then day_sum end)/sum(day_sum) as time_out_rate,sum(case when type='2' then day_cnt end)/sum(day_cnt) as cnt_out_rate
from data_basic.wyd_call_record_date  group by custorm_id,call_time) ds
group by custorm_id,weekday
'''
query=pd.read_sql_query(weekday_mean,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/DayOfWeek.csv',query)

CPU times: user 1min 22s, sys: 734 ms, total: 1min 23s
Wall time: 1min 30s


In [12]:
weekday_mean='''select * from data_basic.wyd_call_record_date'''
query=pd.read_sql_query(weekday_mean,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/wyd_call_record_date.csv',query)

CPU times: user 43min 33s, sys: 11.5 s, total: 43min 44s
Wall time: 50min 50s


###   不同星期

In [14]:
weekday_mean='''select   custorm_id,dayofweek(call_time) as weekday,sum(day_sum)/count(day_sum) as day_sum,sum(day_cnt)/count(day_cnt) as day_cnt,
sum(mean_time)/count(mean_time) as mean_time,
sum(case when type='2' then day_sum end)/sum(day_sum) as day_time_out_rate,
sum(case when type='2' then day_cnt end)/sum(day_cnt) as day_cnt_out_rate
from data_basic.wyd_call_record_date  group  by custorm_id,weekday '''
query=pd.read_sql_query(weekday_mean,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/diff_weekday.csv',query)

CPU times: user 1min 22s, sys: 745 ms, total: 1min 23s
Wall time: 1min 31s


### 在每周不同的日子里打电话的时间上的差异

In [16]:
hourOfDayInWeek='''select custorm_id,weekday,hour_day,sum(day_hour_time)/count(day_hour_time) as hourOfDayTime,
sum(day_hour_cnt)/count(day_hour_cnt) as hourOfDayCnt,sum(day_hour_mean_time)/count(day_hour_mean_time) as hourOfDayMeanTime,
sum(case when type='2' then day_hour_time end)/sum(day_hour_time) as hourOfDayTimeOutRate,
sum(case when type='2' then day_hour_cnt end)/sum(day_hour_cnt) as hourOfDayCntOutRate
from (select custorm_id,weekofyear(call_time) as weekyear,dayofweek(call_time) as weekday,hour(call_time) as hour_day,type,
sum(cast(time as float)) as day_hour_time ,count(time) as day_hour_cnt,sum(cast(time as float))/count(time)  as day_hour_mean_time
from (select * from data_basic.wyd_app_call_record_hive  where cast(time as float)>10 and cast(time as float)<3600 and 
cast(type as float) <3 and  cast(type as float)>=1)d  group by custorm_id,weekyear,weekday,hour_day,type) hs
group by   custorm_id,weekday,hour_day'''
query=pd.read_sql_query(hourOfDayInWeek,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/hourOfDayInWeek.csv',query)

CPU times: user 18min 56s, sys: 8.83 s, total: 19min 5s
Wall time: 20min 47s


## 

In [17]:
sample="select * from  data_basic.wyd_app_call_record_hive where custorm_id='3160062'"
query=pd.read_sql_query(sample,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/sample_relative.csv',query)

CPU times: user 104 ms, sys: 3 ms, total: 107 ms
Wall time: 7.16 s


In [19]:
relative_recognition='''select *  from  data_basic.wyd_summary_stats  h1
join  data_basic.wyd_month_call_stats  h2
on h1.custorm_id =h2.custorm_id and h1.number = h2.number
join data_basic.wyd_isworkday_record h3
on h1.custorm_id =h3.custorm_id and h1.number = h3.number
join (select distinct custorm_id,number,name from data_basic.wyd_app_call_record_hive125) h4
on  h1.custorm_id =h4.custorm_id and h1.number = h4.number'''

query=pd.read_sql_query(relative_recognition,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/relative_recogenition.csv',query)

CPU times: user 25min 44s, sys: 12.9 s, total: 25min 57s
Wall time: 30min 3s


In [21]:
relative_recognition='''select custorm_id,sum(cast(time as float)) as total_time,count(time) as total_freq,avg(cast(time as float)) as avg_duration  from data_basic.wyd_app_call_record_hive125  group by custorm_id'''

query=pd.read_sql_query(relative_recognition,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/relative_summ.csv',query)

CPU times: user 8.32 s, sys: 70 ms, total: 8.39 s
Wall time: 9.03 s


In [26]:
mobi_summary='''select h1.custorm_id,h1.number,h2.name from data_basic.wyd_mobi_call_summary_stats h1 join 
(select distinct custorm_id,number,name from  crawler.app_contacts_hive ) h2 on h1.custorm_id=h2.custorm_id and h1.number=h2.number   where name is not NULL and cast(name  as float) is NULL'''
query=pd.read_sql_query(mobi_summary,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/mobi_summary_name.csv',query)

CPU times: user 3min 12s, sys: 2.32 s, total: 3min 14s
Wall time: 4min 57s


In [7]:
rel_data_sql='''select h1.*,h2.*,h3.* from  (select t1.*,t1.total_duration/t2.total_time as duration_ratio,t1.total_frequency/t2.total_freq as freq_ratio,t1.avg_duration/t2.avg_duration  as avg_ratio from data_basic.wyd_mobi_call_summary_stats  t1 join  data_basic.wyd_mobi_summary_record  t2
on t1.custorm_id=t2.custorm_id) h1 join data_basic.wyd_mobi_call_month_stats h2  on h1.custorm_id=h2.custorm_id and h1.number=h2.number
join data_basic.wyd_mobi_isworkday_record  h3  on h1.custorm_id=h3.custorm_id and h1.number=h3.number
join (select custorm_id,number from data_basic.wyd_relative_used_data)  h5 on h1.custorm_id=h5.custorm_id and h1.number=h5.number'''
query=pd.read_sql_query(rel_data_sql,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/mobi_rel_data.csv',query)

CPU times: user 36min 14s, sys: 18.5 s, total: 36min 33s
Wall time: 38min 6s


In [4]:
mobi_summary='''select * from  data_basic.wyd_mobi_summary_record'''
query=pd.read_sql_query(mobi_summary,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/mobi_cust_summary.csv',query)

CPU times: user 13.7 s, sys: 116 ms, total: 13.8 s
Wall time: 14.9 s


社区发现取数逻辑

In [7]:
comm_detect='''select *  from  (select h1.custorm_id as cust_A,h2.custorm_id as cust_B ,h1.total_frequency  from data_basic.wyd_mobi_call_summary_stats   h1  left join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307 ) h2 
on h1.number =h2.number   where  cast(h1.custorm_id as bigint) != h2.custorm_id) t2 '''
query=pd.read_sql_query(comm_detect,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/comm_detect.csv',query)

CPU times: user 28.5 s, sys: 291 ms, total: 28.8 s
Wall time: 31.7 s


In [9]:
comm_detect='''select h1.custorm_id as cust_A,h2.custorm_id  as cust_B ,h1.call_cnt from data_basic.wyd_mobi_call_record_cnt  h1  left join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307 ) h2 
on h1.phone_contact =h2.number   where  cast(h1.custorm_id as bigint) != h2.custorm_id '''
query=pd.read_sql_query(comm_detect,conn,chunksize=10000)
%time ToDisk('MSN/ymdh/comm_detect_cnt.csv',query)

CPU times: user 1min 40s, sys: 988 ms, total: 1min 41s
Wall time: 1min 55s


##     结构及行为分析

- 通讯录数据

In [11]:
contact='''select h1.custorm_id as cust_A,h2.custorm_id  as cust_B ,h1.call_cnt from data_basic.wyd_mobi_call_record_cnt  h1  left join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307 ) h2 
on h1.phone_contact =h2.number   where  cast(h1.custorm_id as bigint) != h2.custorm_id '''
query=pd.read_sql_query(contact,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/contact_net.csv',query)

CPU times: user 3min 23s, sys: 1.75 s, total: 3min 25s
Wall time: 3min 36s


-  通话记录数据

In [17]:
contact='''select h1.custorm_id as cust_A,h2.custorm_id  as cust_B from (select distinct custorm_id,number  from crawler.app_contacts_hive where  length(custorm_id)<8 )  h1   join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307 ) h2 
on h1.number  =h2.number '''
query=pd.read_sql_query(contact,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/contact_net_call.csv',query)

CPU times: user 2min 24s, sys: 1.43 s, total: 2min 25s
Wall time: 2min 38s


- 短信数据

In [16]:
sms_net='''select h1.custorm_id as cust_A,h2.custorm_id  as cust_B  from (select distinct custorm_id,phone from crawler.app_sms_hive  where length(custorm_id)<8 )  h1  join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307 ) h2 
on h1.phone =h2.number '''
query=pd.read_sql_query(sms_net,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/sms_net.csv',query)

CPU times: user 39.7 s, sys: 433 ms, total: 40.2 s
Wall time: 43.6 s


In [6]:
sms_net='''select  t1.custorm_id ,t2.custorm_id  as custorm_id_B, t1.avg_aday_duration,t1.avg_duration,t1.coef_var,t1.avg_interval,t1.time_engagement_ratio from  
data_basic.wyd_mobi_call_summary_stats  t1 join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307)  t2
on t1.number=t2.number  where  cast(t1.custorm_id  as int)!= t2.custorm_id '''
query=pd.read_sql_query(sms_net,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/id_number_0325.csv',query)

CPU times: user 48.2 s, sys: 564 ms, total: 48.7 s
Wall time: 52.4 s


In [ ]:
sms_net='''select  t1.custorm_id ,t2.custorm_id  as custorm_id_B, t1.avg_aday_duration,t1.avg_duration,t1.coef_var,t1.avg_interval,t1.time_engagement_ratio from  
data_basic.wyd_mobi_call_summary_stats  t1 join 
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307)  t2
on t1.number=t2.number  where  cast(t1.custorm_id  as int)!= t2.custorm_id '''
query=pd.read_sql_query(sms_net,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/id_number_0325.csv',query)

In [7]:
test=pd.read_csv('MSN/chapter1data/id_number_0325.csv',index_col=0)

In [4]:
comm_net='''select  h1.*,h2.custorm_id as cust_b  from 
(select * from data_basic.wyd_community_detection_data  where length(custorm_id) >0) h1 join
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307) h2
on h1.number=h2.number'''
query=pd.read_sql_query(comm_net,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/comm_data.csv',query)

CPU times: user 11min, sys: 5.19 s, total: 11min 5s
Wall time: 11min 49s


In [5]:
comm_net_lable='''select  h1.*,h2.custorm_id as cust_b  from 
(select * from data_basic.wyd_community_data_lable  where length(custorm_id) >0) h1 join
(select acctno as custorm_id, concat(abnor,nor,missing) as number   from data_basic.data_temp_20170307) h2
on h1.number=h2.number'''
query=pd.read_sql_query(comm_net_lable,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/comm_data_name.csv',query)

CPU times: user 3min 47s, sys: 2.08 s, total: 3min 49s
Wall time: 4min 3s


In [4]:
user_info0330='''select distinct id,id_no from dsst.fdl_cust_user_info_chain'''
query=pd.read_sql_query(user_info0330,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/user_info0330.csv',query)

CPU times: user 1min 41s, sys: 1.04 s, total: 1min 42s
Wall time: 1min 52s


###  失恋修复

In [8]:
lost_conn_qury='''select distinct t1.custorm_id,name,number  from  crawler.app_contacts_hive t1 join 
(select distinct  id from data_basic.wyd_lost_conn_info) t2
on  cast(t1.custorm_id as bigint)=t2.id'''
query=pd.read_sql_query(lost_conn_qury,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/lost_conn_info.csv',query)

CPU times: user 3.12 s, sys: 28 ms, total: 3.15 s
Wall time: 7.3 s


In [9]:
lost_conn_qury='''select distinct union_id,id from data_basic.wyd_lost_conn_info'''
query=pd.read_sql_query(lost_conn_qury,conn,chunksize=10000)
%time ToDisk('MSN/chapter1data/lost_conn_map.csv',query)

CPU times: user 47 ms, sys: 0 ns, total: 47 ms
Wall time: 166 ms


In [12]:
a.shape

(927086,)

In [13]:
b.shape

(1870031,)

In [15]:
np.union1d(a,b).shape

(2334512,)